Today, the Office for Budget Responsibility releases its Spring 2025 Economic and Fiscal Outlook. The OBR is an independent body that produces economic and fiscal forecasts for the UK government.

In this report, we'll integrate the OBR's new Spring forecast into PolicyEngine's tax-benefit microsimulation model, and examine the impact this new forecast (against the Autumn 2024 EFO) on the UK household sector. All impacts below are the impact of this forecast change *alone*, excluding any policy measures not already in the baseline policy.

In [1]:
from policyengine import Simulation


# Autumn 2024 OBR Forecast
AUTUMN_24_OBR_FORECAST = {
    "gov.obr.employment_income": {
        "year:2025:1": 1215.6,
        "year:2026:1": 1246.5,
        "year:2027:1": 1277.7,
        "year:2028:1": 1312.9,
        "year:2029:1": 1352.9,
    },
    "gov.obr.mixed_income": {
        "year:2025:1": 177.6,
        "year:2026:1": 196.7,
        "year:2027:1": 205.3,
        "year:2028:1": 214.6,
        "year:2029:1": 225.0,
    },
    "gov.obr.non_labour_income": {
        "year:2025:1": 460.8,
        "year:2026:1": 512.7,
        "year:2027:1": 535.0,
        "year:2028:1": 554.0,
        "year:2029:1": 571.8,
    },
    "gov.obr.consumer_price_index": {
        "year:2025:1": 138.1,
        "year:2026:1": 141.1,
        "year:2027:1": 144.1,
        "year:2028:1": 147.1,
        "year:2029:1": 150.1,
    },
}

# Spring 2025 OBR Forecast - Default to Autumn 2024 values, update these when the new forecast is released
SPRING_25_OBR_FORECAST = {
    "gov.obr.employment_income": {
        "year:2025:1": 1239.8,
        "year:2026:1": 1272.5,
        "year:2027:1": 1305.3,
        "year:2028:1": 1340.7,
        "year:2029:1": 1382.0,
    },
    "gov.obr.mixed_income": {
        "year:2025:1": 202.8,
        "year:2026:1": 210.6,
        "year:2027:1": 218.7,
        "year:2028:1": 227.4,
        "year:2029:1": 237.2,
    },
    "gov.obr.non_labour_income": {
        "year:2025:1": 503.3,
        "year:2026:1": 531.6,
        "year:2027:1": 555.6,
        "year:2028:1": 574.6,
        "year:2029:1": 594.5,
    },
    "gov.obr.consumer_price_index": {
        "year:2025:1": 139.0,
        "year:2026:1": 141.7,
        "year:2027:1": 144.5,
        "year:2028:1": 147.4,
        "year:2029:1": 150.4,
    },
}

# As a test- halve earnings growth params

for year in range(2026, 2030):
    # Calculate growth in autumn forecast
    earnings_growth = AUTUMN_24_OBR_FORECAST["gov.obr.employment_income"][f"year:{year}:1"] - AUTUMN_24_OBR_FORECAST["gov.obr.employment_income"][f"year:{year-1}:1"]
    # Update spring forecast with halved growth
    SPRING_25_OBR_FORECAST["gov.obr.employment_income"][f"year:{year}:1"] = SPRING_25_OBR_FORECAST["gov.obr.employment_income"][f"year:{year-1}:1"] + earnings_growth/2

sim = Simulation(
    country="uk",
    scope="macro",
    baseline=AUTUMN_24_OBR_FORECAST,
    reform=SPRING_25_OBR_FORECAST,
)

from policyengine_core.reforms import Reform
from IPython.display import Markdown
import pandas as pd

baseline = Reform.from_dict(AUTUMN_24_OBR_FORECAST, country_id="uk")
reform = Reform.from_dict(SPRING_25_OBR_FORECAST, country_id="uk")

link = f"https://policyengine.org/uk/policy?reform={reform.api_id}&baseline={baseline.api_id}&region=uk&timePeriod=2029"

Markdown(f"[See the forecast change in PolicyEngine]({link})")

/Users/nikhilwoodruff/policyengine/obr-forecast/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[See the forecast change in PolicyEngine](https://policyengine.org/uk/policy?reform=80226&baseline=80227&region=uk&timePeriod=2029)

## The forecast change

In this report we'll focus on the changes to core economic variables: employment income, mixed income, non-labour income, and inflation. The tables below show the prior and new values for these variables (totals, £ billion).

* The **employment income** forecast by the OBR is used to update wages and salaries in PolicyEngine for future year modelling.
* The **mixed income** forecast by the OBR is used to update self-employment incomes.[^1]
* The **non-labour income** forecast by the OBR is used to update income from savings, investments, and pensions.
* The **inflation** forecast by the OBR is used to update the price level in PolicyEngine, including policy parameters that are indexed to inflation.

[^1]: Per the [OBR's definition](https://obr.uk/forecasts-in-depth/the-economy-forecast/income/#disposable), *"mixed income is largely composed of self-employment income, which is derived from our forecasts of self-employment and whole economy average earnings"*.

In [2]:
# Create a table for the baseline and reform, column 1 is the baseline, column 2 is the reform, column 3 is the change. Years are rows

def add_thousands_separator(x):
    return "{:,}".format(x)

def create_table(baseline, reform):
    years = []
    autumn_values = []
    spring_values = []
    change_values = []
    variables = []
    key_to_label = {
        "gov.obr.employment_income": "Employment income",
        "gov.obr.mixed_income": "Mixed income",
        "gov.obr.non_labour_income": "Non-labour income",
        "gov.obr.consumer_price_index": "Consumer Price Index",
    }

    for year in range(2025, 2030):
        for key in baseline.keys():
            years.append(year)
            variables.append(key_to_label[key])
            autumn_values.append(baseline[key][f"year:{year}:1"])
            spring_values.append(reform[key][f"year:{year}:1"])
            change_values.append(reform[key][f"year:{year}:1"] - baseline[key][f"year:{year}:1"])
    
    df = pd.DataFrame({
        "Year": years,
        "Variable": variables,
        "Autumn 2024": autumn_values,
        "Spring 2025": spring_values,
        "Change": change_values,
    })
    df["Year"] = df["Year"].astype(str)
    df["Autumn 2024"] = df["Autumn 2024"].round(1)
    df["Spring 2025"] = df["Spring 2025"].round(1)
    df["Change"] = df["Change"].round(1)

    df["Autumn 2024"] = df["Autumn 2024"].apply(add_thousands_separator)
    df["Spring 2025"] = df["Spring 2025"].apply(add_thousands_separator)
    df["Change"] = df["Change"].apply(add_thousands_separator)

    return df

reform_table = create_table(AUTUMN_24_OBR_FORECAST, SPRING_25_OBR_FORECAST)

def get_table(key):
    return Markdown(f"### {key}\n\n" + reform_table[reform_table.Variable == key].drop("Variable", axis=1).set_index("Year").to_markdown())

get_table("Employment income")

### Employment income

|   Year | Autumn 2024   | Spring 2025   |   Change |
|-------:|:--------------|:--------------|---------:|
|   2025 | 1,215.6       | 1,239.8       |     24.2 |
|   2026 | 1,246.5       | 1,255.2       |      8.8 |
|   2027 | 1,277.7       | 1,270.8       |     -6.9 |
|   2028 | 1,312.9       | 1,288.4       |    -24.5 |
|   2029 | 1,352.9       | 1,308.4       |    -44.5 |

In [3]:
get_table("Mixed income")

### Mixed income

|   Year |   Autumn 2024 |   Spring 2025 |   Change |
|-------:|--------------:|--------------:|---------:|
|   2025 |         177.6 |         202.8 |     25.2 |
|   2026 |         196.7 |         210.6 |     13.9 |
|   2027 |         205.3 |         218.7 |     13.4 |
|   2028 |         214.6 |         227.4 |     12.8 |
|   2029 |         225   |         237.2 |     12.2 |

In [4]:
get_table("Non-labour income")

### Non-labour income

|   Year |   Autumn 2024 |   Spring 2025 |   Change |
|-------:|--------------:|--------------:|---------:|
|   2025 |         460.8 |         503.3 |     42.5 |
|   2026 |         512.7 |         531.6 |     18.9 |
|   2027 |         535   |         555.6 |     20.6 |
|   2028 |         554   |         574.6 |     20.6 |
|   2029 |         571.8 |         594.5 |     22.7 |

In [5]:
get_table("Consumer Price Index")

### Consumer Price Index

|   Year |   Autumn 2024 |   Spring 2025 |   Change |
|-------:|--------------:|--------------:|---------:|
|   2025 |         138.1 |         139   |      0.9 |
|   2026 |         141.1 |         141.7 |      0.6 |
|   2027 |         144.1 |         144.5 |      0.4 |
|   2028 |         147.1 |         147.4 |      0.3 |
|   2029 |         150.1 |         150.4 |      0.3 |

## Public sector net worth

The table below shows the change in public sector net worth as a result of the forecast change (change to government tax revenue forecast minus spending). Public sector net worth (PSNW), defined by the [OBR](https://obr.uk/box/broader-measures-of-the-public-sector-balance-sheet/), is the broadest National Accounts measure of the public sector balance sheet in the UK.

In the table below, we report *only the change to PSNW implied by the PolicyEngine model*, which is the simulated change in tax revenues less government spending on cash benefits implied by the change in the OBR forecast on the four economic metrics above (excluding debt interest spending).

Table 1: Change in public sector net worth (£ billion)

In [6]:
# First, PSND from forecast changes
psnd_change = []
years = []
for year in range(2025, 2030):
    psnd_baseline = sim.baseline_simulation.calculate("gov_balance", year).sum()/1e9
    psnd_reform = sim.reform_simulation.calculate("gov_balance", year).sum()/1e9
    change = psnd_reform - psnd_baseline
    psnd_change.append(round(change, 1))
    years.append(year)

years.append("2025-2029")
psnd_change.append(round(sum(psnd_change), 1))

Markdown(pd.DataFrame({"Year": years, "PSNW change (£ billions)": psnd_change}).set_index("Year").T.to_markdown())

|                          |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-2029 |
|:-------------------------|-------:|-------:|-------:|-------:|-------:|------------:|
| PSNW change (£ billions) |   28.4 |     11 |    4.1 |   -4.3 |  -13.1 |        26.1 |

In [20]:
Markdown(f"As shown, the OBR forecast update raises the government budget by £{psnd_change[-1]} billion over the period 2025-2029. Per year, this is on average £{round(psnd_change[-1]/5, 1)} billion.")

As shown, the OBR forecast update raises the government budget by £26.1 billion over the period 2025-2029. Per year, this is on average £5.2 billion.

## Poverty rates

The government reports annually on four poverty metrics:

* **Absolute poverty before housing costs**: households with income below 60% of the 2010/11 median income, adjusted for inflation.
* **Absolute poverty after housing costs**: households with income below 60% of the 2010/11 median income after housing costs, adjusted for inflation.
* **Relative poverty before housing costs**: households with income below 60% of the median income in the current year.
* **Relative poverty after housing costs**: households with income below 60% of the median incomeafter housing costs in the current year .

For each of these metrics, we simulate the change in the poverty rate and headcount as a result of the forecast change.

In [8]:
def get_poverty(simulation, year, absolute, bhc, rate):
    population = simulation.calculate("person_id", year).count()
    if absolute:
        if bhc:
            in_poverty = simulation.calculate("in_poverty", year, map_to="person").sum()
        else:
            in_poverty = simulation.calculate("in_poverty_ahc", year, map_to="person").sum()
    else:
        if bhc:
            in_poverty = simulation.calculate("in_relative_poverty_bhc", year, map_to="person").sum()
        else:
            in_poverty = simulation.calculate("in_relative_poverty_ahc", year, map_to="person").sum()

    if rate:
        return round(in_poverty/population * 100, 1)

    else:
        return round(in_poverty / 1e3)


In [22]:
def get_rate_table(absolute, bhc, i):
    years = []
    poverty_rates_autumn = []
    headcounts_autumn = []
    poverty_rates_spring = []
    headcounts_spring = []
    change_rates = []
    change_headcounts = []
    for year in range(2025, 2030):
        years.append(year)
        poverty_rates_autumn.append(get_poverty(sim.baseline_simulation, year, absolute, bhc, rate=True))
        poverty_rates_spring.append(get_poverty(sim.reform_simulation, year, absolute, bhc, rate=True))
        headcounts_autumn.append(get_poverty(sim.baseline_simulation, year, absolute, bhc, rate=False))
        headcounts_spring.append(get_poverty(sim.reform_simulation, year, absolute, bhc, rate=False))
        change_rates.append(round(poverty_rates_spring[-1] - poverty_rates_autumn[-1], 1))
        change_headcounts.append(round(headcounts_spring[-1] - headcounts_autumn[-1], 0))
    
    df = pd.DataFrame({
        "Year": years,
        "Autumn EFO rate (%)": poverty_rates_autumn,
        "Autumn EFO headcount (thousands)": headcounts_autumn,
        "Spring EFO rate (%)": poverty_rates_spring,
        "Spring EFO headcount (thousands)": headcounts_spring,
        "Change in rate (pp)": change_rates,
        "Change in headcount (thousands)": change_headcounts,
    })

    for col in df.columns:
        if "thousands" in col:
            df[col] = df[col].apply(add_thousands_separator)

    title = "Absolute poverty" if absolute else "Relative poverty"
    title += " before housing costs" if bhc else " after housing costs"

    return Markdown(f"Table {i}: {title}\n\n" + df.set_index("Year").to_markdown())

get_rate_table(absolute=True, bhc=True, i=2)

Table 2: Absolute poverty before housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  12.5 | 9,125                              |                  12.3 | 8,961                              |                  -0.2 |                              -164 |
|   2026 |                  12.3 | 9,025                              |                  12.2 | 8,922                              |                  -0.1 |                              -103 |
|   2027 |                  12.3 | 9,113                              |                  12.2 | 9,048                              |                  -0.1 |                               -65 |
|   2028 |                  12.3 | 9,101                              |                  12.3 | 9,098                              |                   0   |                                -3 |
|   2029 |                  12.4 | 9,206                              |                  12.4 | 9,221                              |                   0   |                                15 |

In [23]:
get_rate_table(absolute=True, bhc=False, i=3)

Table 3: Absolute poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  16.1 | 11,713                             |                  15.6 | 11,372                             |                  -0.5 |                              -341 |
|   2026 |                  16.1 | 11,777                             |                  15.6 | 11,448                             |                  -0.5 |                              -329 |
|   2027 |                  16.2 | 11,980                             |                  16   | 11,810                             |                  -0.2 |                              -170 |
|   2028 |                  16.3 | 12,082                             |                  16.1 | 11,974                             |                  -0.2 |                              -108 |
|   2029 |                  16.4 | 12,196                             |                  16.3 | 12,135                             |                  -0.1 |                               -61 |

In [24]:
get_rate_table(absolute=False, bhc=False, i=4)

Table 4: Relative poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  21.4 | 15,569                             |                  21   | 15,312                             |                  -0.4 |                              -257 |
|   2026 |                  21.2 | 15,562                             |                  21   | 15,399                             |                  -0.2 |                              -163 |
|   2027 |                  21.3 | 15,757                             |                  20.9 | 15,426                             |                  -0.4 |                              -331 |
|   2028 |                  21.3 | 15,821                             |                  21   | 15,552                             |                  -0.3 |                              -269 |
|   2029 |                  20.9 | 15,548                             |                  20.9 | 15,541                             |                   0   |                                -7 |

In [25]:
get_rate_table(absolute=False, bhc=False, i=5)

Table 5: Relative poverty after housing costs

|   Year |   Autumn EFO rate (%) | Autumn EFO headcount (thousands)   |   Spring EFO rate (%) | Spring EFO headcount (thousands)   |   Change in rate (pp) |   Change in headcount (thousands) |
|-------:|----------------------:|:-----------------------------------|----------------------:|:-----------------------------------|----------------------:|----------------------------------:|
|   2025 |                  21.4 | 15,569                             |                  21   | 15,312                             |                  -0.4 |                              -257 |
|   2026 |                  21.2 | 15,562                             |                  21   | 15,399                             |                  -0.2 |                              -163 |
|   2027 |                  21.3 | 15,757                             |                  20.9 | 15,426                             |                  -0.4 |                              -331 |
|   2028 |                  21.3 | 15,821                             |                  21   | 15,552                             |                  -0.3 |                              -269 |
|   2029 |                  20.9 | 15,548                             |                  20.9 | 15,541                             |                   0   |                                -7 |

## Inequality

We also project income inequality metrics in PolicyEngine's open source model, including the Gini coefficient, the share of income held by the top 10 percent, and the share of income held by the top 1 percent.

In [26]:
def get_inequality_table(metric, i):
    years = []
    autumn_values = []
    spring_values = []
    change_values = []
    change_values_pct = []

    for year in range(2025, 2030):
        if metric == "Gini coefficient":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).gini() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).gini() * 100)
        elif metric == "Top 10% share":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).top_10_pct_share() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).top_10_pct_share() * 100)
        elif metric == "Top 1% share":
            autumn_values.append(sim.baseline_simulation.calculate("equiv_household_net_income", year).top_1_pct_share() * 100)
            spring_values.append(sim.reform_simulation.calculate("equiv_household_net_income", year).top_1_pct_share() * 100)
        years.append(year)
        change_values.append(round(spring_values[-1] - autumn_values[-1], 1))
        change_values_pct.append(round((spring_values[-1] - autumn_values[-1])/autumn_values[-1] * 100, 1))

    df = pd.DataFrame({
        "Year": years,
        "Autumn EFO": autumn_values,
        "Spring EFO": spring_values,
        "Change (pp)": change_values,
        "Change (%)": change_values_pct,
    })

    df["Autumn EFO"] = df["Autumn EFO"].round(1)
    df["Spring EFO"] = df["Spring EFO"].round(1)

    return Markdown(f"Table {i}: {metric}\n\n" + df.set_index("Year").to_markdown())

get_inequality_table("Gini coefficient", i=6)


Table 6: Gini coefficient

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |         35.3 |         35.7 |           0.4 |          1   |
|   2026 |         35.5 |         35.6 |           0.1 |          0.3 |
|   2027 |         35.6 |         35.6 |           0.1 |          0.2 |
|   2028 |         35.6 |         35.7 |           0   |          0.1 |
|   2029 |         35.7 |         35.7 |          -0   |         -0   |

In [27]:
get_inequality_table("Top 10% share", i=7)

Table 7: Top 10% share

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |         27.2 |         27.3 |           0.2 |          0.6 |
|   2026 |         27.5 |         27.6 |           0.1 |          0.5 |
|   2027 |         27.5 |         27.7 |           0.2 |          0.6 |
|   2028 |         27.6 |         27.7 |           0.1 |          0.4 |
|   2029 |         27.8 |         27.8 |          -0   |         -0.1 |

In [28]:
get_inequality_table("Top 1% share", i=8)

Table 8: Top 1% share

|   Year |   Autumn EFO |   Spring EFO |   Change (pp) |   Change (%) |
|-------:|-------------:|-------------:|--------------:|-------------:|
|   2025 |          7.9 |          8.2 |           0.3 |          3.9 |
|   2026 |          8.2 |          8.3 |           0.1 |          1.6 |
|   2027 |          8.2 |          8.4 |           0.2 |          2   |
|   2028 |          8.3 |          8.5 |           0.2 |          2.9 |
|   2029 |          8.3 |          8.6 |           0.3 |          3.5 |

## Household impacts

The OBR forecast change lowers net income for some households and raises it for others, due to households having different compositions of income sources and spending. The chart below shows an extract of a thousand randomly-sampled households from our enhanced Family Resources Survey-based microdata, with the x-axis showing the household's income percentile in 2025 and the y-axis showing the change in net income in 2030 as a result of the forecast change.

Figure 1: Change in net income by income percentile

In [16]:
from policyengine.utils.charts import *

add_fonts()

In [39]:
df = pd.DataFrame()
df["weight"] = sim.baseline_simulation.calculate("household_weight", 2030).values
df["Gain"] = sim.reform_simulation.calculate("household_net_income", 2030).values - sim.baseline_simulation.calculate("household_net_income", 2030).values
df["Percentile"] = sim.baseline_simulation.calculate("household_net_income", 2030).percentile_rank().values

import plotly.express as px

fig = px.scatter(
    df.sample(weights="weight", n=1_000, random_state=1),
    x="Percentile",
    y="Gain",
    color_discrete_sequence=[BLUE],
    opacity=0.5,
).update_layout(
    title="OBR forecast change impact on households by percentile",
    xaxis_title="Percentile",
    yaxis_title="Change in net income (£)",
    showlegend=False,
    font=dict(family="Roboto Serif"),
    font_color="black",
    yaxis_range=[-5_000, 5_000],
    yaxis_tickformat=",.0f",
    margin={
      "l": 50,
      "r": 50,
      "b": 100,
      "t": 100,
      "pad": 4
    },
    annotations=[
      {
        "x": 1,
        "y": -0.25,
        "xref": "paper",
        "yref": "paper",
        "text": "Source: PolicyEngine UK",
        "showarrow": False,
        "font": {
          "family": "Roboto Serif",
          "size": 10,
          "color": "#616161"
        }
      }
    ],
    images=[
      {
        "source": "/logo512.png",
        "x": 1,
        "y": -0.20,
        "xref": "paper",
        "yref": "paper",
        "sizex": 0.1,
        "sizey": 0.1,
        "xanchor": "right",
        "yanchor": "bottom"
      }
    ],
)

# Add zero line

fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=100,
    y1=0,
    line=dict(
        color=DARK_GRAY,
        width=2,
        dash="solid",
    ),
)

Markdown(f"```plotly\n{fig.to_json()}\n```")

```plotly
{"data":[{"hovertemplate":"Percentile=%{x}\u003cbr\u003eGain=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#2C6496","opacity":0.5,"symbol":"circle"},"mode":"markers","name":"","orientation":"v","showlegend":false,"x":[40.0,80.0,35.0,22.0,16.0,42.0,13.0,92.0,14.0,77.0,80.0,25.0,17.0,42.0,44.0,10.0,61.0,75.0,40.0,10.0,100.0,89.0,40.0,16.0,45.0,69.0,27.0,12.0,18.0,55.0,88.0,89.0,93.0,65.0,93.0,21.0,75.0,99.0,88.0,84.0,57.0,34.0,10.0,29.0,76.0,81.0,68.0,20.0,67.0,83.0,35.0,88.0,36.0,47.0,95.0,39.0,48.0,34.0,8.0,60.0,26.0,57.0,84.0,21.0,41.0,6.0,81.0,50.0,47.0,41.0,65.0,80.0,81.0,85.0,84.0,69.0,51.0,32.0,73.0,12.0,91.0,9.0,91.0,49.0,43.0,28.0,55.0,34.0,58.0,25.0,97.0,10.0,53.0,46.0,92.0,44.0,93.0,60.0,14.0,54.0,81.0,8.0,35.0,89.0,29.0,42.0,14.0,61.0,9.0,45.0,47.0,69.0,79.0,84.0,23.0,73.0,99.0,60.0,82.0,50.0,17.0,74.0,96.0,40.0,6.0,47.0,26.0,19.0,95.0,46.0,51.0,92.0,7.0,95.0,32.0,12.0,90.0,36.0,14.0,81.0,21.0,91.0,23.0,73.0,40.0,15.0,71.0,55.0,39.0,7.0,42.0,90.0,60.0,36.0,3.0,38.0,34.0,10.0,37.0,48.0,58.0,43.0,83.0,30.0,92.0,45.0,29.0,8.0,47.0,52.0,21.0,26.0,87.0,52.0,53.0,58.0,91.0,10.0,89.0,4.0,100.0,68.0,73.0,98.0,50.0,67.0,90.0,22.0,24.0,70.0,56.0,64.0,83.0,57.0,84.0,98.0,87.0,45.0,45.0,89.0,11.0,3.0,46.0,38.0,65.0,18.0,42.0,34.0,64.0,38.0,23.0,89.0,55.0,44.0,2.0,31.0,77.0,44.0,89.0,81.0,6.0,1.0,62.0,59.0,31.0,35.0,46.0,73.0,12.0,67.0,35.0,25.0,100.0,6.0,5.0,53.0,79.0,45.0,22.0,5.0,89.0,27.0,35.0,12.0,9.0,4.0,23.0,8.0,91.0,39.0,38.0,3.0,12.0,48.0,85.0,95.0,99.0,83.0,94.0,94.0,70.0,76.0,30.0,54.0,13.0,16.0,26.0,3.0,41.0,12.0,5.0,76.0,99.0,87.0,31.0,85.0,83.0,47.0,6.0,47.0,38.0,22.0,68.0,1.0,61.0,35.0,99.0,48.0,77.0,89.0,17.0,55.0,24.0,38.0,100.0,82.0,63.0,54.0,26.0,100.0,69.0,61.0,17.0,66.0,21.0,24.0,34.0,74.0,100.0,23.0,94.0,42.0,67.0,67.0,74.0,51.0,49.0,53.0,51.0,63.0,49.0,36.0,83.0,41.0,8.0,58.0,45.0,43.0,55.0,51.0,1.0,64.0,35.0,45.0,35.0,63.0,80.0,25.0,46.0,29.0,95.0,50.0,78.0,84.0,3.0,16.0,67.0,85.0,90.0,66.0,91.0,86.0,96.0,83.0,87.0,65.0,81.0,73.0,23.0,54.0,42.0,78.0,68.0,25.0,59.0,91.0,46.0,17.0,19.0,39.0,10.0,38.0,72.0,47.0,69.0,3.0,20.0,51.0,61.0,36.0,10.0,85.0,3.0,73.0,83.0,80.0,39.0,53.0,8.0,50.0,18.0,47.0,63.0,44.0,8.0,59.0,54.0,82.0,96.0,89.0,32.0,70.0,30.0,84.0,85.0,32.0,96.0,79.0,64.0,58.0,51.0,100.0,58.0,37.0,66.0,78.0,17.0,80.0,70.0,41.0,93.0,43.0,64.0,46.0,53.0,87.0,95.0,72.0,88.0,78.0,58.0,4.0,90.0,49.0,54.0,5.0,15.0,85.0,11.0,97.0,42.0,86.0,85.0,47.0,34.0,1.0,85.0,29.0,51.0,76.0,37.0,11.0,31.0,50.0,78.0,12.0,37.0,21.0,49.0,4.0,60.0,32.0,43.0,68.0,94.0,74.0,79.0,100.0,4.0,65.0,35.0,76.0,69.0,23.0,64.0,39.0,72.0,46.0,6.0,87.0,90.0,4.0,51.0,12.0,50.0,45.0,7.0,67.0,100.0,47.0,38.0,12.0,25.0,96.0,85.0,86.0,45.0,92.0,84.0,54.0,35.0,32.0,93.0,85.0,98.0,88.0,15.0,92.0,54.0,12.0,47.0,33.0,66.0,91.0,18.0,3.0,77.0,87.0,63.0,75.0,8.0,85.0,52.0,63.0,70.0,84.0,25.0,97.0,92.0,82.0,56.0,66.0,36.0,87.0,54.0,40.0,20.0,7.0,77.0,60.0,36.0,24.0,46.0,94.0,55.0,48.0,14.0,6.0,42.0,78.0,17.0,49.0,1.0,11.0,86.0,53.0,99.0,12.0,95.0,89.0,98.0,67.0,71.0,51.0,60.0,87.0,70.0,72.0,45.0,53.0,75.0,13.0,59.0,99.0,59.0,18.0,92.0,83.0,53.0,27.0,80.0,86.0,82.0,25.0,6.0,63.0,100.0,29.0,71.0,86.0,60.0,69.0,97.0,95.0,26.0,72.0,49.0,94.0,46.0,29.0,71.0,2.0,47.0,62.0,58.0,98.0,5.0,51.0,44.0,89.0,94.0,35.0,30.0,35.0,85.0,41.0,82.0,14.0,29.0,49.0,65.0,31.0,79.0,52.0,98.0,38.0,49.0,62.0,93.0,52.0,73.0,34.0,56.0,64.0,61.0,81.0,69.0,74.0,73.0,100.0,35.0,38.0,32.0,6.0,30.0,77.0,43.0,85.0,38.0,52.0,89.0,42.0,44.0,82.0,5.0,61.0,72.0,92.0,63.0,87.0,78.0,65.0,81.0,39.0,66.0,47.0,27.0,91.0,74.0,98.0,44.0,68.0,58.0,51.0,38.0,66.0,73.0,72.0,96.0,77.0,89.0,6.0,50.0,52.0,36.0,71.0,93.0,12.0,86.0,30.0,50.0,72.0,81.0,36.0,70.0,76.0,24.0,82.0,71.0,17.0,97.0,36.0,95.0,1.0,95.0,64.0,34.0,80.0,1.0,6.0,56.0,50.0,61.0,29.0,95.0,80.0,60.0,63.0,64.0,88.0,97.0,98.0,96.0,50.0,37.0,68.0,11.0,68.0,34.0,78.0,74.0,82.0,49.0,81.0,52.0,4.0,40.0,50.0,54.0,90.0,12.0,42.0,92.0,53.0,50.0,44.0,100.0,78.0,60.0,41.0,50.0,29.0,98.0,24.0,66.0,47.0,43.0,60.0,66.0,37.0,100.0,1.0,35.0,100.0,71.0,54.0,4.0,51.0,91.0,17.0,67.0,50.0,53.0,73.0,71.0,45.0,43.0,8.0,10.0,33.0,45.0,46.0,25.0,76.0,45.0,56.0,84.0,24.0,79.0,46.0,85.0,14.0,53.0,24.0,71.0,94.0,34.0,93.0,55.0,35.0,79.0,70.0,47.0,2.0,47.0,83.0,59.0,73.0,11.0,44.0,97.0,60.0,13.0,24.0,40.0,91.0,14.0,3.0,95.0,38.0,78.0,75.0,10.0,46.0,1.0,49.0,79.0,79.0,74.0,48.0,25.0,82.0,64.0,90.0,52.0,47.0,92.0,98.0,83.0,77.0,3.0,79.0,36.0,34.0,13.0,45.0,88.0,89.0,99.0,61.0,97.0,34.0,35.0,14.0,54.0,69.0,38.0,83.0,61.0,64.0,60.0,5.0,3.0,51.0,17.0,3.0,34.0,6.0,46.0,41.0,36.0,90.0,55.0,50.0,43.0,13.0,44.0,61.0,6.0,40.0,58.0,50.0,65.0,49.0,45.0,54.0,88.0,95.0,49.0,40.0,23.0,20.0,4.0,83.0,89.0,25.0,38.0,96.0,97.0,9.0,36.0,78.0,74.0,12.0,83.0,86.0,26.0,2.0,58.0,70.0,70.0,100.0,5.0,23.0,1.0,34.0,54.0,4.0,1.0,10.0,3.0,58.0,89.0,69.0,97.0,90.0,91.0,12.0,87.0,12.0,72.0,17.0,80.0,51.0,83.0,40.0,72.0,83.0,74.0,98.0,64.0,94.0,95.0,74.0,42.0,2.0,52.0,25.0,18.0,87.0,99.0,5.0,58.0,20.0,15.0,33.0,62.0,51.0,90.0,87.0,49.0,41.0,59.0,32.0,68.0,94.0,73.0,87.0,88.0,3.0,11.0,85.0,40.0,41.0,5.0,53.0,99.0,86.0,42.0,16.0,96.0,8.0,61.0,62.0,41.0,8.0,4.0,86.0,46.0,12.0,18.0,9.0,72.0,75.0,99.0,32.0,38.0,41.0,75.0,76.0],"xaxis":"x","y":[1010.0,-2590.0,115.0,-381.0,-252.0,-1323.0,74.0,-3249.0,445.0,-475.0,-2242.0,-184.0,184.0,-1381.0,-1344.0,646.0,-313.0,-1211.0,-479.0,-130.0,3795.0,-2727.0,-458.0,-4.0,-363.0,1466.0,18.0,71.0,-732.0,-1543.0,-3038.0,356.0,-887.0,-646.0,-1702.0,356.0,-2440.0,5617.0,-2178.0,-597.0,-2005.0,519.0,69.0,-59.0,1573.0,1701.0,657.0,323.0,-1744.0,-2613.0,7.0,-3201.0,158.0,-1668.0,-3649.0,120.0,728.0,452.0,-44.0,-942.0,52.0,-701.0,-278.0,-166.0,145.0,-558.0,-2938.0,378.0,-977.0,1521.0,1341.0,-2883.0,-1225.0,-2354.0,-2600.0,-109.0,-129.0,531.0,1334.0,-5.0,-2867.0,-521.0,-1903.0,-1089.0,-65.0,-34.0,-420.0,92.0,-1114.0,351.0,-5201.0,-512.0,-1207.0,1177.0,-2316.0,825.0,-490.0,-2287.0,-10.0,928.0,-1488.0,4.0,579.0,2531.0,-19.0,-1287.0,639.0,-1724.0,-275.0,464.0,-1472.0,-891.0,-2564.0,-2823.0,-67.0,-1465.0,-3911.0,-1485.0,-2949.0,94.0,-4.0,-1806.0,-2144.0,59.0,68.0,513.0,173.0,-280.0,-4173.0,-185.0,-239.0,-2444.0,280.0,-4058.0,463.0,-403.0,-3363.0,660.0,-258.0,1900.0,598.0,2069.0,-2.0,756.0,1822.0,78.0,-1923.0,-152.0,114.0,54.0,499.0,-2429.0,-295.0,308.0,-1376.0,703.0,-1037.0,205.0,7.0,814.0,-581.0,-1391.0,-2698.0,-291.0,-3438.0,714.0,396.0,246.0,-385.0,-337.0,-1371.0,523.0,-2466.0,-510.0,-47.0,-1337.0,-2816.0,59.0,-2810.0,-29.0,5615.0,-694.0,-2035.0,-5238.0,1388.0,1651.0,2353.0,-1.0,-28.0,-2573.0,-1246.0,1252.0,-2625.0,-1626.0,-1262.0,-1746.0,-2980.0,430.0,858.0,-2471.0,-195.0,-61.0,-57.0,290.0,328.0,-96.0,-1246.0,-119.0,-558.0,-710.0,-49.0,-1008.0,21.0,1278.0,637.0,-1489.0,767.0,521.0,-2101.0,-2696.0,-46.0,-1.0,-555.0,-1625.0,562.0,-64.0,-209.0,2201.0,74.0,-1947.0,1128.0,150.0,4437.0,103.0,73.0,797.0,-1272.0,-66.0,128.0,-75.0,2457.0,51.0,-32.0,101.0,-102.0,-8.0,-44.0,-639.0,-3871.0,96.0,-348.0,-139.0,87.0,-1205.0,-1660.0,-2330.0,-2052.0,-2638.0,-4018.0,-3041.0,-1499.0,-2675.0,-926.0,-1346.0,-289.0,74.0,-904.0,-4.0,-163.0,-295.0,75.0,-579.0,-4792.0,-1649.0,-26.0,-35.0,-2568.0,847.0,52.0,438.0,-166.0,-30.0,482.0,-6.0,-1855.0,-1061.0,-6656.0,36.0,-597.0,-2379.0,-16.0,88.0,41.0,-102.0,43622.0,-1491.0,769.0,-5973.0,-51.0,-8206.0,525.0,-1871.0,-2.0,-387.0,16.0,-44.0,495.0,-2467.0,8811.0,-1.0,-953.0,1014.0,-1452.0,-1323.0,1715.0,214.0,821.0,-705.0,208.0,-1859.0,-1411.0,-36.0,-2643.0,-1254.0,165.0,-13.0,-1418.0,864.0,1198.0,-184.0,3.0,-83.0,-268.0,-1412.0,-114.0,-2030.0,-735.0,44.0,-1349.0,-55.0,-3699.0,-539.0,-2645.0,-953.0,-209.0,38.0,-1703.0,1808.0,1091.0,-1895.0,-2683.0,843.0,-6349.0,73.0,1946.0,-1614.0,-2616.0,-2356.0,-22.0,-525.0,-17.0,2109.0,-1214.0,465.0,-428.0,-3070.0,651.0,-327.0,112.0,-1198.0,-492.0,-44.0,930.0,-1534.0,-1933.0,-30.0,-961.0,901.0,1207.0,-73.0,560.0,-2609.0,82.0,-540.0,-1909.0,-2538.0,102.0,-509.0,168.0,-1615.0,310.0,-104.0,704.0,492.0,170.0,971.0,-1138.0,-2004.0,-1594.0,2383.0,-19.0,1766.0,-58.0,-2735.0,-3022.0,-86.0,3664.0,-2580.0,1452.0,-2028.0,-116.0,-10440.0,-594.0,-1128.0,-1516.0,-1340.0,-715.0,-2445.0,-2097.0,35.0,-3704.0,726.0,-2197.0,-1378.0,-1444.0,-2013.0,-3258.0,-1805.0,-2420.0,-2378.0,-867.0,-58.0,-3113.0,-542.0,-575.0,-16.0,-18.0,-2960.0,297.0,-3769.0,273.0,1902.0,-2880.0,490.0,-1043.0,223.0,-2739.0,-946.0,-1436.0,-2599.0,679.0,-73.0,47.0,-865.0,-2289.0,218.0,-230.0,-52.0,903.0,-2.0,1610.0,-1091.0,1310.0,2235.0,-3212.0,-1797.0,-2469.0,4539.0,-17.0,-875.0,-1067.0,-2503.0,-945.0,22.0,-1019.0,451.0,-977.0,-1556.0,350.0,-234.0,-2785.0,-4.0,-1022.0,436.0,-1595.0,309.0,-30.0,-963.0,18460.0,-978.0,599.0,-2.0,-830.0,-3053.0,-2932.0,-198.0,-1359.0,-2964.0,2248.0,9.0,-389.0,564.0,-3378.0,-2329.0,-5447.0,-3328.0,307.0,1384.0,-1907.0,-2.0,-1535.0,288.0,-1708.0,-3299.0,24.0,-44.0,-2210.0,-3351.0,-1486.0,-1770.0,73.0,-2533.0,705.0,-1388.0,-2110.0,-3184.0,493.0,-2178.0,-2588.0,669.0,-165.0,-1594.0,41.0,-2868.0,-1691.0,693.0,797.0,75.0,-2493.0,-1334.0,-449.0,0.0,-161.0,-3066.0,-194.0,-1534.0,199.0,73.0,-144.0,-2627.0,134.0,993.0,-663.0,183.0,-3182.0,-1879.0,1668.0,-34.0,-2996.0,-2864.0,-4693.0,989.0,1342.0,-1524.0,-1469.0,-2833.0,-2203.0,1357.0,-1415.0,-1441.0,-1551.0,186.0,345.0,-4454.0,819.0,-310.0,-3691.0,-2052.0,1135.0,-80.0,1951.0,-861.0,-2643.0,-967.0,67.0,-1485.0,-4995.0,333.0,-1318.0,-2816.0,-699.0,-2186.0,-3454.0,-524.0,-46.0,-2082.0,87.0,-3629.0,-1407.0,-912.0,-1299.0,501.0,853.0,-50.0,-1569.0,1126.0,79.0,-501.0,29.0,-1523.0,-3953.0,-607.0,-20.0,-135.0,-3170.0,-998.0,-2751.0,-129.0,581.0,-114.0,-2192.0,-49.0,-2955.0,-1747.0,-6356.0,-124.0,802.0,-1499.0,-320.0,-520.0,1519.0,-123.0,1504.0,-1133.0,-1936.0,-1087.0,-2385.0,-2465.0,-2386.0,5516.0,1071.0,-810.0,-57.0,107.0,-23.0,-598.0,897.0,-2088.0,-71.0,502.0,-2716.0,-1314.0,-1366.0,772.0,75.0,-465.0,-182.0,-3279.0,1613.0,-3016.0,-2622.0,-1377.0,1532.0,1019.0,900.0,-38.0,172.0,-2144.0,-2432.0,-3065.0,-1379.0,-2121.0,-1795.0,-476.0,622.0,-1638.0,-1728.0,1302.0,161.0,-1832.0,-1231.0,64.0,-2078.0,-1787.0,-1126.0,406.0,-3291.0,541.0,475.0,-59.0,-820.0,-1740.0,-3191.0,727.0,-1629.0,-1187.0,-53.0,-1942.0,-2327.0,-33.0,-4528.0,-247.0,-1559.0,-1.0,-2186.0,-565.0,-136.0,-482.0,1.0,487.0,-454.0,-1601.0,159.0,-85.0,-3465.0,-2781.0,1196.0,502.0,-1770.0,-3086.0,-3745.0,-6122.0,-4214.0,-1859.0,17.0,-2173.0,53.0,2161.0,-1071.0,-2678.0,-1006.0,-2589.0,-1231.0,853.0,-1479.0,-33.0,738.0,-1494.0,1356.0,-3364.0,75.0,1239.0,-3554.0,-1849.0,-1554.0,-598.0,-4012.0,1845.0,-1834.0,-1285.0,-1636.0,156.0,-5737.0,113.0,-2189.0,-705.0,-654.0,950.0,-1854.0,-1028.0,19980.0,-320.0,595.0,4711.0,-2328.0,869.0,-14.0,610.0,-2788.0,-57.0,-1871.0,-1621.0,-1755.0,-2497.0,-2212.0,182.0,541.0,320.0,111.0,373.0,-1393.0,-382.0,-855.0,-2436.0,-109.0,-257.0,-2718.0,-57.0,-1312.0,-371.0,-2768.0,294.0,-1810.0,198.0,-2297.0,-3357.0,671.0,-3611.0,-1863.0,339.0,-525.0,-2210.0,-1434.0,2.0,-1468.0,-2736.0,1418.0,-983.0,-98.0,653.0,688.0,-1641.0,164.0,-810.0,-599.0,-2741.0,-42.0,-48.0,-4223.0,941.0,429.0,-778.0,183.0,-878.0,-2.0,-1555.0,-2766.0,-2289.0,-1923.0,275.0,363.0,-2872.0,1196.0,-3250.0,1308.0,1105.0,-2847.0,-5545.0,-2464.0,-2761.0,335.0,-1044.0,713.0,-1086.0,-210.0,-1195.0,-3120.0,-3121.0,-6793.0,-1749.0,865.0,379.0,-41.0,258.0,2044.0,-162.0,650.0,276.0,-1286.0,1071.0,-1845.0,-28.0,-15.0,806.0,-1.0,-46.0,-35.0,215.0,-1414.0,-1014.0,-449.0,-3401.0,1078.0,-1646.0,510.0,354.0,864.0,-1415.0,54.0,-68.0,-1467.0,-526.0,969.0,-1004.0,-1389.0,-192.0,-710.0,-2411.0,-151.0,-23.0,339.0,152.0,-504.0,-2657.0,-3162.0,-61.0,-1112.0,-2312.0,-3082.0,-508.0,588.0,-2744.0,-1533.0,-262.0,-2565.0,-3525.0,-937.0,123.0,-1896.0,-122.0,-2324.0,22921.0,110.0,200.0,-290.0,605.0,239.0,-69.0,1142.0,371.0,72.0,1497.0,-2902.0,-2268.0,-5419.0,-3166.0,-1081.0,-558.0,-3337.0,135.0,-234.0,266.0,-1618.0,-59.0,754.0,-274.0,828.0,-2006.0,-1783.0,4931.0,-1974.0,-3597.0,-3035.0,2367.0,1055.0,-25.0,-1756.0,-43.0,-749.0,-3464.0,4210.0,-1.0,-1381.0,-3.0,861.0,-1101.0,-2012.0,-1606.0,-3153.0,-2736.0,60.0,27.0,674.0,413.0,1051.0,2982.0,-1588.0,-3291.0,-3374.0,-190.0,-606.0,-2761.0,-1392.0,-1256.0,-526.0,1194.0,-6384.0,-2000.0,323.0,-64.0,-3263.0,-70.0,-1849.0,-1317.0,439.0,-49.0,-17.0,2964.0,-1143.0,234.0,654.0,188.0,-1198.0,-1174.0,223.0,-28.0,90.0,-1282.0,-257.0,-372.0],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Percentile"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Change in net income (\u00a3)"},"range":[-5000,5000],"tickformat":",.0f"},"legend":{"tracegroupgap":0},"margin":{"t":100,"l":50,"r":50,"b":100,"pad":4},"font":{"family":"Roboto Serif","color":"black"},"title":{"text":"OBR forecast change impact on households by percentile"},"showlegend":false,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine UK","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}],"shapes":[{"line":{"color":"#616161","dash":"solid","width":2},"type":"line","x0":0,"x1":100,"y0":0,"y1":0}]}}
```

## Impact by income decile

The chart below shows the relative change in income held by each 2030 household income decile, as a result of the change in the OBR forecast.

Figure 2: Change in income by income decile

In [18]:
decile = sim.baseline_simulation.calculate("household_income_decile", 2030)
income = sim.baseline_simulation.calculate("household_net_income", 2030)
gain = sim.reform_simulation.calculate("household_net_income", 2030) - sim.baseline_simulation.calculate("household_net_income", 2030)
aggregate_change_by_decile = gain.groupby(decile).sum()
relative_change_by_decile = gain.groupby(decile).sum() / income.groupby(decile).sum()

df = pd.DataFrame({
    "Aggregate change": aggregate_change_by_decile.values,
    "Relative change": relative_change_by_decile.values,
    "Income decile": aggregate_change_by_decile.index,
})
df = df[df["Income decile"] > 0]
df["Text"] = df["Relative change"].apply(lambda x: f"{x:.1%}")

fig = px.bar(
    df,
    x="Income decile",
    y="Relative change",
    text="Text",
    color_discrete_sequence=[DARK_GRAY],
    labels={"Aggregate change": "Change in net income (£)", "Income decile": "Income decile"},
    title="OBR forecast change impact on 2030 net income by income decile",
).update_layout(
    font=dict(family="Roboto Serif"),
    yaxis_tickformat=".0%",
    xaxis_title="Income decile",
    xaxis_tickvals=list(range(1, 11)),
    showlegend=False,
    font_color="black",
    margin={
      "l": 50,
      "r": 50,
      "b": 100,
      "t": 100,
      "pad": 4
    },
    annotations=[
      {
        "x": 1,
        "y": -0.25,
        "xref": "paper",
        "yref": "paper",
        "text": "Source: PolicyEngine UK",
        "showarrow": False,
        "font": {
          "family": "Roboto Serif",
          "size": 10,
          "color": "#616161"
        }
      }
    ],
    images=[
      {
        "source": "/logo512.png",
        "x": 1,
        "y": -0.20,
        "xref": "paper",
        "yref": "paper",
        "sizex": 0.1,
        "sizey": 0.1,
        "xanchor": "right",
        "yanchor": "bottom"
      }
    ],
)
Markdown(f"```plotly\n{fig.to_json()}\n```")

```plotly
{"data":[{"alignmentgroup":"True","hovertemplate":"Income decile=%{x}\u003cbr\u003eRelative change=%{y}\u003cbr\u003eText=%{text}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#616161","pattern":{"shape":""}},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"text":["0.0%","-0.3%","-0.1%","-0.6%","-1.6%","-1.5%","-2.0%","-1.5%","-2.2%","-0.6%"],"textposition":"auto","x":[1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[0.0004025890092378338,-0.002824166415140995,-0.0010594049697834288,-0.005573731685334178,-0.016444473378985205,-0.015447508798184547,-0.01956110180368262,-0.015197629023774382,-0.022164616900411035,-0.006219302211171338],"yaxis":"y","type":"bar"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Income decile"},"tickvals":[1,2,3,4,5,6,7,8,9,10]},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Relative change"},"tickformat":".0%"},"legend":{"tracegroupgap":0},"title":{"text":"OBR forecast change impact on 2030 net income by income decile"},"barmode":"relative","font":{"family":"Roboto Serif","color":"black"},"margin":{"l":50,"r":50,"b":100,"t":100,"pad":4},"showlegend":false,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine UK","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

## Households by beneficiary status

The ONS [reports](https://www.ons.gov.uk/peoplepopulationandcommunity/personalandhouseholdfinances/incomeandwealth/bulletins/theeffectsoftaxesandbenefitsonhouseholdincome/financialyearending2023) on its self-defined net recipient rate. This is the percentage of households that receive more in (cash and in-kind) benefits than they pay in taxes.

The table below shows the change in a similar measure, defined as the percentage of households that receive more in cash benefits than they pay in taxes, as a result of the forecast change.

In [30]:
years = []
net_recipients_rate_a = []
net_recipients_rate_s = []
net_recipients_people_a = []
net_recipients_people_s = []
change_rate = []
change_people = []

for year in range(2025, 2030):
    years.append(year)
    net_recipients_rate_a.append((sim.baseline_simulation.calculate("household_benefits", year) > sim.baseline_simulation.calculate("household_tax", year)).mean())
    net_recipients_rate_s.append((sim.reform_simulation.calculate("household_benefits", year) > sim.reform_simulation.calculate("household_tax", year)).mean())
    change_rate.append(net_recipients_rate_s[-1] - net_recipients_rate_a[-1])
    net_recipients_people_a.append((sim.baseline_simulation.calculate("household_benefits", year, map_to="person") > sim.baseline_simulation.calculate("household_tax", year, map_to="person")).sum())
    net_recipients_people_s.append((sim.reform_simulation.calculate("household_benefits", year, map_to="person") > sim.reform_simulation.calculate("household_tax", year, map_to="person")).sum())
    change_people.append(net_recipients_people_s[-1] - net_recipients_people_a[-1])
df = pd.DataFrame({
    "Year": years,
    "Net recipients rate (%) (Autumn 2024)": net_recipients_rate_a,
    "Net recipients rate (%) (Spring 2025)": net_recipients_rate_s,
    "Change in rate (pp)": change_rate,
    "Net recipients (Autumn 2024) (m)": net_recipients_people_a,
    "Net recipients (Spring 2025) (m)": net_recipients_people_s,
    "Change in net recipients (k)": change_people,
})

df["Net recipients rate (%) (Autumn 2024)"] = df["Net recipients rate (%) (Autumn 2024)"].apply(lambda x: f"{x*100:.1f}")
df["Net recipients rate (%) (Spring 2025)"] = df["Net recipients rate (%) (Spring 2025)"].apply(lambda x: f"{x*100:.1f}")
df["Change in rate (pp)"] = df["Change in rate (pp)"].apply(lambda x: f"{x*100:.1f}")

df["Net recipients (Autumn 2024) (m)"] = df["Net recipients (Autumn 2024) (m)"].apply(lambda x: f"{x/1e6:,.1f}")
df["Net recipients (Spring 2025) (m)"] = df["Net recipients (Spring 2025) (m)"].apply(lambda x: f"{x/1e6:,.1f}")
df["Change in net recipients (k)"] = df["Change in net recipients (k)"].apply(lambda x: f"{x/1e3:,.0f}")


Markdown(f"Table 9: Net recipients by forecast \n\n" + df.set_index("Year").to_markdown())

Table 9: Net recipients by forecast 

|   Year |   Net recipients rate (%) (Autumn 2024) |   Net recipients rate (%) (Spring 2025) |   Change in rate (pp) |   Net recipients (Autumn 2024) (m) |   Net recipients (Spring 2025) (m) |   Change in net recipients (k) |
|-------:|----------------------------------------:|----------------------------------------:|----------------------:|-----------------------------------:|-----------------------------------:|-------------------------------:|
|   2025 |                                    45.4 |                                    44.9 |                  -0.5 |                               30.2 |                               29.8 |                           -462 |
|   2026 |                                    45.3 |                                    45   |                  -0.3 |                               30.5 |                               30.2 |                           -301 |
|   2027 |                                    45.1 |                                    44.9 |                  -0.2 |                               30.6 |                               30.4 |                           -166 |
|   2028 |                                    44.9 |                                    45   |                   0   |                               30.5 |                               30.6 |                             77 |
|   2029 |                                    44.8 |                                    44.9 |                   0.1 |                               30.5 |                               30.7 |                            158 |